# EBSI Wallet Conformance (Natural Person)

> EBSI v2 DID for Natural Person(s) is introduced. 

## Setup conformance headers

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

# str(uuid.uuid4())

headers = {
    "Conformance": "d830d790-752e-4ad4-a946-d0fb034e01d8",
}

## Setup wallet

In [2]:
from ebsi_wallet.main import wallet

# Setup wallet
client = await wallet("init_v2")


## ISSUE_011

In [3]:
from ebsi_wallet.main import conformance
from ebsi_wallet.util import parse_query_string_parameters_from_url

options = {
    "conformance": headers.get("Conformance"),
    "flow_type": "cross-device",
    "credential_type": "pda1"
}

result = await conformance("issuerInitiate", headers=headers, options=options)

console.log("issuerInitiate", result)

session_token = result.replace("openid://", "")
credential_type = parse_query_string_parameters_from_url(
    session_token).get("credential_type")[0]
issuer_url = parse_query_string_parameters_from_url(
    session_token).get("issuer")[0]

console.log("credential_type", credential_type)

options = {
    "client": client,
    "credential_type" : credential_type
}

result = await conformance("issuerAuthorize", options=options)
console.log("issuerAuthorize", result)


[02:15:33] issuerInitiate                                                                          ]8;id=389459;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1074265329.py\1074265329.py]8;;\:]8;id=338080;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1074265329.py#12\12]8;;\
           openid://initiate_issuance?issuer=https%3A%2F%2Fapi-conformance.ebsi.eu%2Fconformance%2                 
           Fv2&credential_type=https%3A%2F%2Fapi-conformance.ebsi.eu%2Ftrusted-schemas-registry%2F                 
           v2%2Fschemas%2FzGUR7RxCZtwQFSdwow63xF8hP2gMtH55incKsADwrcwzZ&conformance=d830d790-752e-                 
           4ad4-a946-d0fb034e01d8                                                                                  

           credential_type                                                                         ]8;id=559732;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1074265329.py\1074265329.py]8;;\:]8;id=896104;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1074265329.py#20\20]8;;\
           https://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/zGUR7RxCZtwQFSdwow6                 
           3xF8hP2gMtH55incKsADwrcwzZ                                                                              

[02:15:34] issuerAuthorize                                                                         ]8;id=401591;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1074265329.py\1074265329.py]8;;\:]8;id=65496;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1074265329.py#28\28]8;;\
           {'state': '7e0bb4ecebd0', 'code': 'e9921f4946e361553d08'}                                               

## ISSUE_021

In [4]:
options = {
    "did_version": client.did_version,
    "code": result["code"]
}

result = await conformance("issuerToken", options=options, headers=headers)
console.log("issuerToken", result)


           issuerToken                                                                              ]8;id=95876;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1256846934.py\1256846934.py]8;;\:]8;id=513193;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/1256846934.py#7\7]8;;\
           {                                                                                                       
               'token_type': 'Bearer',                                                                             
               'access_token': '77b7b1b6c95e7507572e',                                                             
               'expires_in': 3600,                                                                                 
               'c_nonce': '64ca569cb047ccac8ae4',                                                                  
               'id_token':                                                                                         
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NksifQ.eyJpYXQiOjE2ODM5MjQzMzQsInN1YiI6Imh0dHBzOi8vbG9j                
           YWxob3N0OjMwMDAiLCJhdWQiOiJodHRwczovL2xvY2FsaG9zdDozMDAwIiwiaXNzIjoiZGlkOmVic2k6emNmY3dH                
           akxCb2pjenc5eWhtVUZFM1oifQ.RDKlUm0bJMw9L00zfy_2qpQsk0IQOcCQ5O-VJkmCA8JJ43KhJZkGvwu90EsWm                
           73pViF0gwHkmdT3LZzNbwDE2g'                                                                              
           }                                                                                                       

## ISSUE_031

In [5]:
from ebsi_wallet.did_jwt import decode_jwt

options = {
    "c_nonce": result["c_nonce"],
    "client": client,
    "issuer_url": issuer_url,
    "credential_type": credential_type,
}

access_token = result["access_token"]

headers = {
    "Authorization": f"Bearer {access_token}",
    "Conformance": headers["Conformance"]
}

result = await conformance("issuerCredential", options=options, headers=headers)
console.log("issuerCredential", result)

issue_031_credential = result["credential"]

credential = decode_jwt(result["credential"])

console.log("Received credential", credential)

[02:15:35] issuerCredential                                                                         ]8;id=609804;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/608560569.py\608560569.py]8;;\:]8;id=263393;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/608560569.py#18\18]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NksiLCJ0eXAiOiJKV1QiLCJraWQiOiJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkU                
           zWiNrZXlzLTEifQ.eyJqdGkiOiJ1cm46Y3JlZGVudGlhbDpybnR1bzpjZWYxMDljYy1jNjhlLTQ1ZTgtOThlNi1h                
           MjcyZjk2YmMzNjgiLCJzdWIiOiJkaWQ6ZWJzaTp6ZmV6ckFCeVBTRjU3Wmg1WWh5OGJFWVZjOVJKS0tvcjVacFNq                
           VllmVmNGTTciLCJpc3MiOiJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkUzWiIsIm5iZiI6MTY4MzkyNDMz                
           NSwiZXhwIjoxOTk5NTQzNTM1LCJpYXQiOjE2ODM5MjQzMzUsInZjIjp7IkBjb250ZXh0IjpbImh0dHBzOi8vd3d3                
           LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIl0sImlkIjoidXJuOmNyZWRlbnRpYWw6cm50dW86Y2VmMTA5Y2Mt                
           YzY4ZS00NWU4LTk4ZTYtYTI3MmY5NmJjMzY4IiwidHlwZSI6WyJWZXJpZmlhYmxlQ3JlZGVudGlhbCIsIlZlcmlm                
           aWFibGVBdHRlc3RhdGlvbiIsIlZlcmlmaWFibGVQb3J0YWJsZURvY3VtZW50QTEiXSwiaXNzdWVyIjoiZGlkOmVi                
           c2k6emNmY3dHakxCb2pjenc5eWhtVUZFM1oiLCJpc3N1YW5jZURhdGUiOiIyMDIzLTA1LTEyVDIwOjQ1OjM1WiIs                
           InZhbGlkRnJvbSI6IjIwMjMtMDUtMTJUMjA6NDU6MzVaIiwiaXNzdWVkIjoiMjAyMy0wNS0xMlQyMDo0NTozNVoi                
           LCJjcmVkZW50aWFsU3ViamVjdCI6eyJzZWN0aW9uMSI6eyJwZXJzb25hbElkZW50aWZpY2F0aW9uTnVtYmVyIjoi                
           MSIsInNleCI6IjAxIiwic3VybmFtZSI6IkRhbHRvbiIsImZvcmVuYW1lcyI6IkpvZSBKYWNrIFdpbGxpYW0gQXZl                
           cmVsbCIsImRhdGVCaXJ0aCI6IjE5ODUtMDgtMTUiLCJuYXRpb25hbGl0aWVzIjpbIkJFIl0sInN0YXRlT2ZSZXNp                
           ZGVuY2VBZGRyZXNzIjp7InN0cmVldE5vIjoic3NzLCBubm4gIiwicG9zdENvZGUiOiJwcHAiLCJ0b3duIjoiY2Nj                
           IiwiY291bnRyeUNvZGUiOiJCRSJ9LCJzdGF0ZU9mU3RheUFkZHJlc3MiOnsic3RyZWV0Tm8iOiJzc3MsIG5ubiAi                
           LCJwb3N0Q29kZSI6InBwcCIsInRvd24iOiJjY2MiLCJjb3VudHJ5Q29kZSI6IkJFIn19LCJzZWN0aW9uMiI6eyJt                
           ZW1iZXJTdGF0ZVdoaWNoTGVnaXNsYXRpb25BcHBsaWVzIjoiREUiLCJzdGFydGluZ0RhdGUiOiIyMDIyLTEwLTA5                
           IiwiZW5kaW5nRGF0ZSI6IjIwMjItMTAtMjkiLCJjZXJ0aWZpY2F0ZUZvckR1cmF0aW9uQWN0aXZpdHkiOnRydWUs                
           ImRldGVybWluYXRpb25Qcm92aXNpb25hbCI6ZmFsc2UsInRyYW5zaXRpb25SdWxlc0FwcGx5QXNFQzg4MzIwMDQi                
           OmZhbHNlfSwic2VjdGlvbjMiOnsicG9zdGVkRW1wbG95ZWRQZXJzb24iOmZhbHNlLCJlbXBsb3llZFR3b09yTW9y                
           ZVN0YXRlcyI6ZmFsc2UsInBvc3RlZFNlbGZFbXBsb3llZFBlcnNvbiI6dHJ1ZSwic2VsZkVtcGxveWVkVHdvT3JN                
           b3JlU3RhdGVzIjp0cnVlLCJjaXZpbFNlcnZhbnQiOnRydWUsImNvbnRyYWN0U3RhZmYiOmZhbHNlLCJtYXJpbmVy                
           IjpmYWxzZSwiZW1wbG95ZWRBbmRTZWxmRW1wbG95ZWQiOmZhbHNlLCJjaXZpbEFuZEVtcGxveWVkU2VsZkVtcGxv                
           eWVkIjp0cnVlLCJmbGlnaHRDcmV3TWVtYmVyIjpmYWxzZSwiZXhjZXB0aW9uIjpmYWxzZSwiZXhjZXB0aW9uRGVz                
           Y3JpcHRpb24iOiIiLCJ3b3JraW5nSW5TdGF0ZVVuZGVyMjEiOmZhbHNlfSwic2VjdGlvbjQiOnsiZW1wbG95ZWUi                
           OmZhbHNlLCJzZWxmRW1wbG95ZWRBY3Rpdml0eSI6dHJ1ZSwibmFtZUJ1c2luZXNzTmFtZSI6IjEiLCJyZWdpc3Rl                
           cmVkQWRkcmVzcyI6eyJzdHJlZXRObyI6IjEsIDEgMSIsInBvc3RDb2RlIjoiMSIsInRvd24iOiIxIiwiY291bnRy                
           eUNvZGUiOiJERSJ9fSwic2VjdGlvbjUiOnsibm9GaXhlZEFkZHJlc3MiOnRydWV9LCJzZWN0aW9uNiI6eyJuYW1l                
           IjoiTmF0aW9uYWwgSW5zdGl0dXRlIGZvciB0aGUgU29jaWFsIFNlY3VyaXR5IG

           Received credential                                                                      ]8;id=596477;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/608560569.py\608560569.py]8;;\:]8;id=918483;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/608560569.py#24\24]8;;\
           {                                                                                                       
               'header': {                                                                                         
                   'alg': 'ES256K',                                                                                
                   'typ': 'JWT',                                                                                   
                   'kid': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z#keys-1'                                                
               },                                                                                                  
               'payload': {                                                                                        
                   'jti': 'urn:credential:rntuo:cef109cc-c68e-45e8-98e6-a272f96bc368',                             
                   'sub': 'did:ebsi:zfezrAByPSF57Zh5Yhy8bEYVc9RJKKor5ZpSjVYfVcFM7',                                
                   'iss': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z',                                                      
                   'nbf': 1683924335,                                                                              
                   'exp': 1999543535,                                                                              
                   'iat': 1683924335,                                                                              
                   'vc': {                                                                                         
                       '@context': ['https://www.w3.org/2018/credentials/v1'],                                     
                       'id': 'urn:credential:rntuo:cef109cc-c68e-45e8-98e6-a272f96bc368',                          
                       'type': [                                                                                   
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiablePortableDocumentA1'                                                          
                       ],                                                                                          
                       'issuer': 'did:ebsi:zcfcwGjLBojczw9yhmUFE3Z',                                               
                       'issuanceDate': '2023-05-12T20:45:35Z',                                                     
                       'validFrom': '2023-05-12T20:45:35Z',                                                        
                       'issued': '2023-05-12T20:45:35Z',                                                           
                       'credentialSubject': {                                                                      
                           'section1': {                                                                           
                               'personalIdentificationNumber': '1',                                                
                               'sex': '01',                                                                        
                               'surname': 'Dalton',                                                                
                               'forenames': 'Joe Jack William Averell',                                            
                               'dateBirth': '1985-08-15',                

## VERIFY_011

In [6]:
headers = {
    "Conformance": headers.get("Conformance")
}
options = {
    "did_version": "v2"
}

result = await conformance("verifierAuthRequest", options=options, headers=headers)
console.log("verifierAuthRequest", result)

[02:15:36] verifierAuthRequest                                                                      ]8;id=213063;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/3364399777.py\3364399777.py]8;;\:]8;id=92240;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/3364399777.py#9\9]8;;\
           {                                                                                                       
               'client_id':                                                                                        
           'https://api-conformance.ebsi.eu/conformance/v2/verifier-mock/authentication-responses',                
               'response_type': 'id_token',                                                                        
               'scope': 'openid',                                                                                  
               'claims':                                                                                           
           '{"id_token":{"email":null},"vp_token":{"presentation_definition":{"id":"conformance_moc                
           k_vp_request","input_descriptors":[{"id":"conformance_mock_vp","name":"Conformance Mock                 
           VP","purpose":"Only accept a VP containing a Conformance Mock                                           
           VA","constraints":{"fields":[{"path":["$.vc.credentialSchema"],"filter":{"allOf":[{"type                
           ":"array","contains":{"type":"object","properties":{"id":{"type":"string","pattern":"htt                
           ps://api-conformance.ebsi.eu/trusted-schemas-registry/v2/schemas/z3kRpVjUFj4Bq8qHRENUHiZ                
           rVF5VgMBUe7biEafp1wf2J"}},"required":["id"]}}]}}]}}],"format":{"jwt_vp":{"alg":["ES256K"                
           ]}}}}}',                                                                                                
               'redirect_uri':                                                                                     
           'https://api-conformance.ebsi.eu/conformance/v2/verifier-mock/authentication-responses',                
               'nonce': 'c9a3fe7e-eeca-4573-b4f1-cec89d84247a'                                                     
           }                                                                                                       

### Create Verifiable Presentation JWT

In [7]:
from ebsi_wallet.main import compute

encoded_credential = issue_031_credential
credential_issuer = credential['payload']['iss']

options = {
    "credential": encoded_credential,
    "client": client,
    "issuer": credential_issuer,
    "audience": credential_issuer
}

jwt_vp = await compute("createPresentationJwt", options=options)
console.log("createPresentationJwt", jwt_vp)

           createPresentationJwt                                                                   ]8;id=757301;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/2803278071.py\2803278071.py]8;;\:]8;id=57968;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/2803278071.py#14\14]8;;\
           {                                                                                                       
               'jwtVp':                                                                                            
           'eyJhbGciOiAiRVMyNTZLIiwgInR5cCI6ICJKV1QiLCAia2lkIjogImRpZDplYnNpOnpmZXpyQUJ5UFNGNTdaaD                 
           VZaHk4YkVZVmM5UkpLS29yNVpwU2pWWWZWY0ZNNyNQbGdBWjVycHB0VFZMZGxlNk1nSEVTb2RNcXJadC12NnRaN                 
           F9VcmNwNno0IiwgImp3ayI6IHsia3R5IjogIkVDIiwgImNydiI6ICJzZWNwMjU2azEiLCAieCI6ICJOZVotUVVn                 
           ZUtsWUtjNEMzMnBuYUxCOGlOVVowOGg1NmxTUmVvaFF3WHlnIiwgInkiOiAiYVBVWGlDSWhaX0VOWmlINmlqX2V                 
           RNmtvSnF6YUJOdnFlMzBHZEgxRXVxOCJ9fQ.eyJqdGkiOiAidXJuOmRpZDpmM2E3N2ZiYy0zMDc4LTQ0NzgtYTI                 
           3NC1lOWU4M2UyNjFjYmMiLCAic3ViIjogImRpZDplYnNpOnpmZXpyQUJ5UFNGNTdaaDVZaHk4YkVZVmM5UkpLS2                 
           9yNVpwU2pWWWZWY0ZNNyIsICJpc3MiOiAiZGlkOmVic2k6emZlenJBQnlQU0Y1N1poNVloeThiRVlWYzlSSktLb                 
           3I1WnBTalZZZlZjRk03IiwgIm5iZiI6IDE2ODM5MjQzMjksICJleHAiOiAxNjgzOTI1MjM2LCAiaWF0IjogMTY4                 
           MzkyNDMzNiwgImF1ZCI6ICJkaWQ6ZWJzaTp6Y2Zjd0dqTEJvamN6dzl5aG1VRkUzWiIsICJ2cCI6IHsiaWQiOiA                 
           idXJuOmRpZDpmM2E3N2ZiYy0zMDc4LTQ0NzgtYTI3NC1lOWU4M2UyNjFjYmMiLCAiQGNvbnRleHQiOiBbImh0dH                 
           BzOi8vd3d3LnczLm9yZy8yMDE4L2NyZWRlbnRpYWxzL3YxIl0sICJ0eXBlIjogWyJWZXJpZmlhYmxlUHJlc2Vud                 
           GF0aW9uIl0sICJob2xkZXIiOiAiZGlkOmVic2k6emZlenJBQnlQU0Y1N1poNVloeThiRVlWYzlSSktLb3I1WnBT                 
           alZZZlZjRk03IiwgInZlcmlmaWFibGVDcmVkZW50aWFsIjogWyJleUpoYkdjaU9pSkZVekkxTmtzaUxDSjBlWEF                 
           pT2lKS1YxUWlMQ0pyYVdRaU9pSmthV1E2WldKemFUcDZZMlpqZDBkcVRFSnZhbU42ZHpsNWFHMVZSa1V6V2lOcl                 
           pYbHpMVEVpZlEuZXlKcWRHa2lPaUoxY200NlkzSmxaR1Z1ZEdsaGJEcHliblIxYnpwalpXWXhNRGxqWXkxak5qa                 
           GxMVFExWlRndE9UaGxOaTFoTWpjeVpqazJZbU16TmpnaUxDSnpkV0lpT2lKa2FXUTZaV0p6YVRwNlptVjZja0ZD                 
           ZVZCVFJqVTNXbWcxV1doNU9HSkZXVlpqT1ZKS1MwdHZjalZhY0ZOcVZsbG1WbU5HVFRjaUxDSnBjM01pT2lKa2F                 
           XUTZaV0p6YVRwNlkyWmpkMGRxVEVKdmFtTjZkemw1YUcxVlJrVXpXaUlzSW01aVppSTZNVFk0TXpreU5ETXpOU3                 
           dpWlhod0lqb3hPVGs1TlRRek5UTTFMQ0pwWVhRaU9qRTJPRE01TWpRek16VXNJblpqSWpwN0lrQmpiMjUwWlhoM                 
           ElqcGJJbWgwZEhCek9pOHZkM2QzTG5jekxtOXlaeTh5TURFNEwyTnlaV1JsYm5ScFlXeHpMM1l4SWwwc0ltbGtJ                 
           am9pZFhKdU9tTnlaV1JsYm5ScFlXdzZjbTUwZFc4NlkyVm1NVEE1WTJNdFl6WTRaUzAwTldVNExUazRaVFl0WVR                 
           JM01tWTVObUpqTXpZNElpd2lkSGx3WlNJNld5SldaWEpwWm1saFlteGxRM0psWkdWdWRHbGhiQ0lzSWxabGNtbG                 
           1hV0ZpYkdWQmRIUmxjM1JoZEdsdmJpSXNJbFpsY21sbWFXRmliR1ZRYjNKMFlXSnNaVVJ2WTNWdFpXNTBRVEVpW                 
           FN3aWFYTnpkV1Z5SWpvaVpHbGtPbVZpYzJrNmVtTm1ZM2RIYWt4Q2IycGplbmM1ZVdodFZVWkZNMW9pTENKcGMz                 
           TjFZVzVqWlVSaGRHVWlPaUl5TURJekxUQTFMVEV5VkRJd09qUTFPak0xV2lJc0luWmhiR2xrUm5KdmJTSTZJakl                 
           3TWpNdE1EVXRNVEpVTWpBNk5EVTZNelZhSWl3aWFYTnpkV1ZrSWpvaU1qQXlNeTB3TlMweE1sUXlNRG8wTlRvek                 
           5Wb2lMQ0pqY21Wa1pXNTBhV0ZzVTNWaWFtVmpkQ0k2ZXlKelpXTjBhVzl1TVNJNmV5SndaWEp6YjI1aGJFbGtaV                 
           zUwYVdacFkyRjBhVzl1VG5WdFltVnlJam9pTVNJc0luTmxlQ0k2SWpBeElpd2ljM1Z5Ym1GdFpTSTZJa1JoYkhS                 
           dmJpSXNJbVp2Y21WdVlXMWxjeUk2SWtwdlpTQktZV05ySUZkcGJHeHBZVzBnUVhabGNtVnNiQ0lzSW1SaGRHVkN                 
           hWEowYUNJNklqRTVPRFV0TURndE1UVWlMQ0p1WVhScGIyNWhiR2wwYVdWeklq

## VERIFY_031

In [8]:
options = {
    "credential": encoded_credential,
    "jwtVp": jwt_vp["jwtVp"],
    "client": client
}

vp_status = await conformance("verifierAuthResponse", options=options, headers={})
console.log("Verifiable presentation status", vp_status)

[02:15:39] Verifiable presentation status                                                           ]8;id=903413;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/3868489859.py\3868489859.py]8;;\:]8;id=720463;file:///var/folders/61/8pv2r1d505lb_yy2410br0pw0000gn/T/ipykernel_6785/3868489859.py#8\8]8;;\
           {                                                                                                       
               'result': True,                                                                                     
               'validations': {                                                                                    
                   'vpFormat': {'status': True},                                                                   
                   'presentation': {'status': True},                                                               
                   'credential': {'status': True}                                                                  
               }                                                                                                   
           }                                                                                                       